In [1]:
%pip install earthengine-api



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 127.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.0/299.0 kB 132.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 93.3 kB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 kB 140.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 91.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.7/173.7 kB 140.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 130.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 141.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 kB 130.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 189.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 126.5 kB/s eta 0:00:0000:0100:01
Note: yo

In [19]:
import os
import ee

# Set your project working directory (adjust if needed)
os.chdir("/home/kuldeep-chamoli/Desktop/GIS/GIS")

# Authenticate (only if needed)
# ee.Authenticate()

# Initialize EE client
ee.Initialize()

print("Earth Engine initialized!")


Earth Engine initialized!


In [20]:
import geopandas as gpd
import json

gdf = gpd.read_file("data/raw/bengaluru_boundary.geojson")

# Convert to EE geometry
coords = json.loads(gdf.to_json())["features"][0]["geometry"]
boundary = ee.Geometry.Polygon(coords["coordinates"])

boundary


ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              77.4600507,
              12.9050674
            ],
            [
              77.4601312,
              12.9047651
            ],
            [
              77.4601386,
              12.9041089
            ],
            [
              77.4601282,
              12.9030417
            ],
            [
              77.4601564,
              12.901633
            ],
            [
              77.4602001,
              12.9003869
            ],
            [
              77.4603159,
              12.9001947
            ],
            [
              77.4607135,
              12.9000297
            ],
            [
              77.4606848,
              12.8993286
            ],
            [
              77.4606767,
              12.8984986
            ],
            [
         

In [21]:
# Landsat-8 Surface Reflectance Collection (Tier 1)
collection = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(boundary)
    .filterDate("2024-01-01", "2024-12-31")
    .map(lambda img: img.updateMask(img.select('QA_PIXEL').bitwiseAnd(1<<3).eq(0)))  # cloud mask
)

# Median composite
image = collection.median()

# Convert DN to reflectance
def scale_l8(img):
    return img.multiply(0.0000275).add(-0.2)

image_ref = scale_l8(image.select(["SR_B4", "SR_B5"]))  # Red, NIR

# Compute NDVI
ndvi = image_ref.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")

ndvi


In [22]:
# Quick preview image (PNG)
url_thumb = ndvi.getThumbURL({
    'min': -1,
    'max': 1,
    'region': boundary,
    'dimensions': 512
})
url_thumb


'https://earthengine.googleapis.com/v1/projects/626140348690/thumbnails/fd18090f060acce2f79081ba4cf8a20d-ce78d1d97fbddd9aabdb4a57630f0185:getPixels'

In [23]:
# Quick preview image (PNG)
url_thumb = ndvi.getThumbURL({
    'min': -1,
    'max': 1,
    'region': boundary,
    'dimensions': 512
})
url_thumb


'https://earthengine.googleapis.com/v1/projects/626140348690/thumbnails/41dcf62238335b07a66bea6d3a6c0d08-c0f1326212563997491897f8c481a172:getPixels'

In [24]:
download_url = ndvi.clip(boundary).getDownloadURL({
    "scale": 30,                # Landsat resolution
    "crs": "EPSG:4326",         # Lat/Lon
    "region": boundary,         # Clip to city boundary
    "format": "GEO_TIFF"
})

download_url


'https://earthengine.googleapis.com/v1/projects/626140348690/thumbnails/7d96f6984778a849e81d7baeea577235-c3a5cc66fce7aad83358ddfe4af51e81:getPixels'

In [25]:
print("Download NDVI using the above URL. Then place it at:")
print("data/raw/sentinel_ndvi/ndvi_bengaluru_2024.tif")


Download NDVI using the above URL. Then place it at:
data/raw/sentinel_ndvi/ndvi_bengaluru_2024.tif
